# PyTorch Tutorial 02. Pytorch by Example

- 基本的に，[このチュートリアル](https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html)の内容に基づいている．  

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
np.random.seed(1234) ## fix the random values

import torch

########################################
## 各種の定数を定義する
########################################

# N: バッチサイズ ; D_in: 入力の次元;
# H: 隠し層の数   ; D_out: 出力の次元.
N, D_in, H, D_out = 64, 1000, 100, 10
learning_rate = 1e-6 # 学習率

########################################
## 乱数生成データ
########################################

# 入力と出力データを生成
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# ネットワークの重みを初期化する乱数
w1_init = np.random.randn(D_in, H)
w2_init = np.random.randn(H   , D_out)

## NumPy

NumPyによるFully connected neural networkの実装例

- 3 layers = 隠し層は１つ
- 活性化関数はランプ(ReLU)

In [2]:
# -*- coding: utf-8 -*-
w1 = w1_init.copy()
w2 = w2_init.copy()

for t in range(500):
    # 正方向の計算でyを予測
    h = x.dot(w1)
    h_relu = np.maximum(h, 0) ## Ramp function
    y_pred = h_relu.dot(w2)

    # lossの計算
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # lossに従い，w1とw2を計算するための逆伝搬
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # アップデート
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 733.6940576155048
199 6.704564966949013
299 0.1012884202711603
399 0.0017032859556660963
499 2.970849663274526e-05


## PyTorch

- NumPyでの実装と同様な振る舞いをPyTorch(nn無し)で再現  
- ソースではPyTorchも乱数初期化しているが，なるべく似た同様な結果が出るように  
  初期値もNumPyから変換した値を使うように変えた．


In [3]:
# -*- coding: utf-8 -*-
dtype  = torch.float
device = torch.device("cpu")
#device = torch.device("cuda:0") # GPUでの実行する場合

# NumPyで生成した乱数入力と出力データを変換
x_pt = torch.tensor(x, device=device, dtype=dtype)
y_pt = torch.tensor(y, device=device, dtype=dtype)

# ネットワークの重みをNumPyで生成した乱数で初期化
w1_pt = torch.tensor(w1_init, device=device, dtype=dtype)
w2_pt = torch.tensor(w2_init, device=device, dtype=dtype)

for t in range(500):
    
    # 正方向の計算でyを予測
    h = x_pt.mm(w1_pt)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2_pt)

    # lossの計算
    loss = (y_pred - y_pt).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # lossに従い，w1とw2を計算するための逆伝搬
    grad_y_pred = 2.0 * (y_pred - y_pt)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2_pt.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x_pt.t().mm(grad_h)

    # アップデート
    w1_pt -= learning_rate * grad_w1
    w2_pt -= learning_rate * grad_w2

99 733.6937255859375
199 6.704724311828613
299 0.10128504037857056
399 0.0020254584960639477
499 0.00016310632054228336


## Autograd

### PyTorch: Tensor and Autograd

- 最新の複雑なニューラルネットワークを上記例のように自前で実装するのはかなり難しい．  
- そこで，Autograd機能を利用した実装は次のようになる．まだtorch.nnは用いない．

In [ ]:
# -*- coding: utf-8 -*-

# NumPyで生成した乱数入力と出力データを変換 (再度)
# これらの数値は定数であり，勾配計算の必要性を表す
# requires_gradは基本値のFalseのままにおいておく．
x_pt = torch.tensor(x, device=device, dtype=dtype)
y_pt = torch.tensor(y, device=device, dtype=dtype)
print(x_pt.requires_grad, y_pt.requires_grad) ## requires_grad: False as default

# 重みを保存するために，乱数で生成されたTensorを変換して取り入れる．
# requires_grad=True に設定することで，逆伝搬途中に
# これらのテンソルに対する勾配を計算する必要を示す
w1_pt = torch.tensor(w1, device=device, dtype=dtype, requires_grad=True)
w2_pt = torch.tensor(w2, device=device, dtype=dtype, requires_grad=True)
print(w1_pt.requires_grad, w2_pt.requires_grad)

for t in range(500):
    
    # feedforward: テンソル演算に基づき，yの予想値を計算する．
    # 前述した単純にテンソルを用いた例とあまり変わらないが，
    # 逆伝搬を別途実装しなくても同時に計算されるのが長所と言える．
    y_pred = x_pt.mm(w1_pt).clamp(min=0).mm(w2_pt)

    # 損失関数の計算
    # スカラー値(=中身)へのアクセスはloss.item()で行う
    loss = (y_pred - y_pt).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 自動微分で逆伝搬を計算する．
    # requires_grad=True になっている全てのテンソルについて勾配が計算される．
    # 計算された結果もテンソルとしてw1_pt.gradとw2_pt.gradに保存される．
    loss.backward()

    # 最急降下法(gradient descent)で重みを手動で更新する．
    # 自動微分による履歴記録が必要ないため，torch.no_grad()を適用する．
    with torch.no_grad():
        w1_pt -= learning_rate * w1_pt.grad
        w2_pt -= learning_rate * w2_pt.grad

        # 勾配バッファをクリアする
        w1_pt.grad.zero_()
        w2_pt.grad.zero_()

    # 上記の作業は，torch.optim.SGDによって大体される

False False
True True
99 1.322716343565844e-05
199 8.224774319387507e-06
299 5.967043762211688e-06
399 4.655492830352159e-06
499 3.837288204522338e-06


### PyTorch: Autograd for user-defined functions

In [ ]:
# -*- coding: utf-8 -*-

class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Functionを継承するユーザ定義の自動微分可能な関数．
    テンソルのfeedforwardとbackpropを実装する．
    """

    @staticmethod
    def forward(ctx, input):
        """
        順伝搬では，入力テンソルを受け取り，それの出力を返す．
        ctxはコンテキストオブジェクトであり，逆伝搬のための情報保持に用いる．
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환합니다.
        ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에
        사용합니다. ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 어떠한
        객체도 저장(cache)해 둘 수 있습니다.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        逆伝搬の段階では，出力に対する損失の勾配関数のテンソルを受け取り，
        入力に対する損失関数の勾配を計算する
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용합니다.
    # 여기에 'relu'라는 이름을 붙였습니다.
    relu = MyReLU.apply

    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산합니다;
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산합니다.
    y_pred = relu(x.mm(w1)).mm(w2)

    # 손실을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograde를 사용하여 역전파 단계를 계산합니다.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 갱신합니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()


### torch.nn

Pre-defined functional layers for neural networks

In [ ]:
# -*- coding: utf-8 -*-

# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의합니다.
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성합니다. 각각의 Linear Module은 선형 함수를 사용하여
# 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있습니다;
# 여기에서는 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용하겠습니다.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다. Module 객체는
    # __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 합니다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다. 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듭니다.
    model.zero_grad()

    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를
    # 계산합니다. 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때
    # Tensor 내에 저장되므로, 이 호출은 모든 모델의 모든 학습 가능한 매개변수의
    # 변화도를 계산하게 됩니다.
    loss.backward()

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다. 각 매개변수는
    # Tensor이므로 이전에 했던 것과 같이 변화도에 접근할 수 있습니다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


### torch.optim: 最適化モジュール

In [ ]:
# -*- coding: utf-8 -*-
import torch

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의합니다.
# 여기서는 Adam을 사용하겠습니다; optim 패키지는 다른 다양한 최적화 알고리즘을
# 포함하고 있습니다. Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지
# 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듭니다. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고)
    # 누적되기 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를
    # 참조하세요.
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()

    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

### torch.nn.Module

User-defined functional layers for neural networks

In [ ]:
# -*- coding: utf-8 -*-
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 2개의 nn.Linear 모듈을 생성하고, 멤버 변수로 지정합니다.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Tensor를 받고 출력 데이터의 Tensor를
        반환해야 합니다. Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한
        Module도 사용할 수 있습니다.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞에서 정의한 클래스를 생성하여 모델을 구성합니다.
model = TwoLayerNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 만듭니다. SGD 생성자에 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함됩니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### PyTorch: フロー制御 + 重みの共有

In [ ]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 순전파 단계에서 사용할 3개의 nn.Linear 인스턴스를 생성합니다.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        모델의 순전파 단계에서, 무작위로 0, 1, 2 또는 3 중에 하나를 선택하고
        은닉층을 계산하기 위해 여러번 사용한 middle_linear Module을 재사용합니다.

        각 순전파 단계는 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를
        정의할 때 반복문이나 조건문과 같은 일반적인 Python 제어 흐름 연산자를 사용할
        수 있습니다.

        여기에서 연산 그래프를 정의할 때 동일 Module을 여러번 재사용하는 것이
        완벽히 안전하다는 것을 알 수 있습니다. 이것이 각 Module을 한 번씩만 사용할
        수 있었던 Lua Torch보다 크게 개선된 부분입니다.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞서 정의한 클래스를 생성(instantiating)하여 모델을 구성합니다.
model = DynamicNet(D_in, H, D_out)

# 손실함수와 Optimizer를 만듭니다. 이 이상한 모델을 순수한 확률적 경사 하강법
# (stochastic gradient decent)으로 학습하는 것은 어려우므로, 모멘텀(momentum)을
# 사용합니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(end)